In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV, train_test_split, cross_validate
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
import pickle

import tqdm
from tqdm import tqdm_notebook
from multiprocessing.dummy import Pool
from IPython.display import clear_output

# Load data

In [2]:
mushroom_data = pd.read_csv("agaricus-lepiota.data", header=None)

column_names = ["classes", "cap-shape", "cap-surface", "cap-color", "bruises?", "odor", "gill-attachment",
                "gill-spacing", "gill-size", "gill-color", "stalk-shape", "stalk-root", "stalk-surface-above-ring",
                "stalk-surface-below-ring", "stalk-color-above-ring", "stalk-color-below-ring", "veil-type", 
                "veil-color", "ring-number", "ring-type", "spore-print-color", "population", "habitat"]
mushroom_data.columns = column_names

In [3]:
mushroom_data.head()

,classes,cap-shape,cap-surface,cap-color,bruises?,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [4]:
mushroom_data.dtypes

classes                     object
cap-shape                   object
cap-surface                 object
cap-color                   object
bruises?                    object
odor                        object
gill-attachment             object
gill-spacing                object
gill-size                   object
gill-color                  object
stalk-shape                 object
stalk-root                  object
stalk-surface-above-ring    object
stalk-surface-below-ring    object
stalk-color-above-ring      object
stalk-color-below-ring      object
veil-type                   object
veil-color                  object
ring-number                 object
ring-type                   object
spore-print-color           object
population                  object
habitat                     object
dtype: object

# Data preparation

In [5]:
for column in column_names:
    mushroom_data[column] = mushroom_data[column].astype('category')
    mushroom_data[column] = mushroom_data[column].cat.codes

In [6]:
mushroom_data.head()

,classes,cap-shape,cap-surface,cap-color,bruises?,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1


In [7]:
mushroom_data.dtypes

classes                     int8
cap-shape                   int8
cap-surface                 int8
cap-color                   int8
bruises?                    int8
odor                        int8
gill-attachment             int8
gill-spacing                int8
gill-size                   int8
gill-color                  int8
stalk-shape                 int8
stalk-root                  int8
stalk-surface-above-ring    int8
stalk-surface-below-ring    int8
stalk-color-above-ring      int8
stalk-color-below-ring      int8
veil-type                   int8
veil-color                  int8
ring-number                 int8
ring-type                   int8
spore-print-color           int8
population                  int8
habitat                     int8
dtype: object

## Gini by factors

In [8]:
idx_trn, idx_tst = train_test_split(mushroom_data.index, test_size=0.2, random_state=42, 
                                    stratify=mushroom_data[['classes']])

In [9]:
def gini(var):
    df = mushroom_data.copy()
    x_trn = df.loc[idx_trn, var]
    y_trn = df.loc[idx_trn, 'classes']
    x_tst = df.loc[idx_tst, var]
    y_tst = df.loc[idx_tst, 'classes']
    
    if x_trn.dtype in ['O','object']:
        cats = pd.DataFrame({'x': x_trn, 'y': y_trn}).fillna('#NAN#').groupby('x').agg('mean').sort_values('y').index.values
        X_trn = pd.Categorical(x_trn.fillna('#NAN#'), categories=cats, ordered=True).codes.reshape(-1, 1)
        X_tst = pd.Categorical(x_tst.fillna('#NAN#'), categories=cats, ordered=True).codes.reshape(-1, 1)
    else:
        repl = min(x_trn.min(), x_tst.min())-1 if np.isfinite(min(x_trn.min(), x_tst.min())-1) else -999999
        #repl = x_trn.min()-1 if np.isfinite(x_trn.min())-1 else -999999
        X_trn = x_trn.fillna(repl).replace(np.inf, repl).replace(-np.inf, repl).values.reshape(-1, 1)
        X_tst = x_tst.fillna(repl).replace(np.inf, repl).replace(-np.inf, repl).values.reshape(-1, 1)
    
    obvious_gini_trn = 2*roc_auc_score(y_trn, X_trn)-1
    obvious_gini_tst = 2*roc_auc_score(y_tst, X_tst)-1

    if obvious_gini_trn < 0:
        obvious_gini_trn = -obvious_gini_trn
        obvious_gini_tst = -obvious_gini_tst

    parameters = {'min_samples_leaf':[0.01, 0.025, 0.05, 0.1]}
    dt = DecisionTreeClassifier(random_state=1)
    clf = GridSearchCV(dt, parameters, cv=4, scoring='roc_auc', n_jobs=10)
    clf.fit(X_trn, y_trn)

    true_gini_trn = 2*clf.best_score_-1
    true_gini_tst = 2*roc_auc_score(y_tst, clf.predict_proba(X_tst)[:, 1])-1

    if true_gini_trn < 0:
        true_gini_trn = -true_gini_trn
        true_gini_tst = -true_gini_tst

    if obvious_gini_trn > true_gini_trn:
        return [var, obvious_gini_trn, obvious_gini_tst]
    else:
        return [var, true_gini_trn, true_gini_tst]

In [10]:
with Pool(20) as p:
    vars_gini = list(tqdm.tqdm_notebook(p.imap(gini, column_names), total=len(column_names)))

<ipython-input-10-20d11f835e4b>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  vars_gini = list(tqdm.tqdm_notebook(p.imap(gini, column_names), total=len(column_names)))


  0%|          | 0/23 [00:00<?, ?it/s]

In [11]:
vars_gini = pd.DataFrame(vars_gini)
vars_gini.set_index(0, inplace=True)
vars_gini.columns = ['gini_train', 'gini_test']

vars_gini

,gini_train,gini_test
0,,
classes,1.000000,1.000000
cap-shape,0.185225,0.212779
cap-surface,0.180956,0.201118
cap-color,0.207793,0.237278
bruises?,0.495815,0.490001
odor,0.967487,0.969555
gill-attachment,0.040117,0.044683
gill-spacing,0.256251,0.257858
gill-size,0.490563,0.535203


## Correlation analysis

In [12]:
vars_corrs = mushroom_data.loc[:, column_names].corr().abs().stack().reset_index().drop_duplicates()
vars_corrs = vars_corrs[vars_corrs.level_0!=vars_corrs.level_1]
vars_corrs.columns = ['var_1', 'var_2', 'correlation']
vars_corrs = vars_corrs.set_index(['var_1', 'var_2'], drop=True).sort_values(by='correlation', ascending=False)

# write top 10 correlations
vars_corrs.head(10)

,,correlation
var_1,var_2,
veil-color,gill-attachment,0.897518
gill-attachment,veil-color,0.897518
ring-type,bruises?,0.692973
bruises?,ring-type,0.692973
ring-type,gill-color,0.629398
gill-color,ring-type,0.629398
gill-size,spore-print-color,0.622991
spore-print-color,gill-size,0.622991
gill-size,classes,0.540024


In [13]:
vars_drop = []

for v in vars_corrs[vars_corrs.correlation > 0.7].index.values:
    if v[0] not in vars_drop and v[1] not in vars_drop:
        vars_drop.append(v[1] if vars_gini.loc[v[0], 'gini_train'] > vars_gini.loc[v[1], 'gini_train'] else v[0])
        
del v

## Feature selection

In [14]:
# all variables
vars0 = column_names[1:]
print(len(vars0))

# drop values with gini less than 3%
vars1 = [v for v in vars0 if vars_gini.loc[v, 'gini_train'] >= 0.03]
print(len(vars1))

# drop correlated variables
vars2 = [v for v in vars1 if v not in vars_drop]
print(len(vars2))

22
21
20


In [15]:
i = 0

for var_lst in [vars0, vars1, vars2]:
    i += 1
    lgb = LGBMClassifier(max_depth=1, n_estimators=250, random_state=42, n_jobs=30)
    
    cv = cross_validate(lgb, mushroom_data.loc[:, var_lst], mushroom_data.loc[:, 'classes'], 
                        cv=5, scoring='roc_auc', n_jobs=20, return_train_score=True)
    
    lgb.fit(mushroom_data[var_lst], mushroom_data['classes'])
    
    print({'Variables': len(var_lst), 
           'Train CV': round(cv['train_score'].mean()*2-1, 4), 
           'Test CV': round(cv['test_score'].mean()*2-1, 4)})

{'Variables': 22, 'Train CV': 0.9993, 'Test CV': 0.8706}
{'Variables': 21, 'Train CV': 0.9993, 'Test CV': 0.8706}
{'Variables': 20, 'Train CV': 0.9993, 'Test CV': 0.8706}


Exclude features which have no feature importances in gradient boosting

In [16]:
var_lst_imp = pd.Series(dict(zip(var_lst, lgb.feature_importances_)))
var_lst = [i for i in var_lst_imp.index if var_lst_imp.loc[i]>0]
print({'exclude': [i for i in var_lst_imp.index if var_lst_imp.loc[i]<=0]})
print(len(var_lst))

{'exclude': ['cap-shape', 'cap-surface', 'bruises?', 'veil-color']}
16


Forward selection procedure

In [17]:
forw_cols = []
current_ginis = pd.Series({'Train CV':0, 'Test CV':0})

def forw(x):
    lgb = LGBMClassifier(max_depth=1, n_estimators=250, random_state=42, n_jobs=1)
    cv = cross_validate(lgb, mushroom_data.loc[:, forw_cols+[x]], mushroom_data.loc[:, 'classes'],
                        cv=5, scoring='roc_auc', n_jobs=1, return_train_score=True)
    lgb.fit(mushroom_data.loc[:, forw_cols+[x]], mushroom_data.loc[:, 'classes'])
    return x, pd.Series({
        'Train CV': cv['train_score'].mean()*2-1,
        'Test CV': cv['test_score'].mean()*2-1
    })

forwards_log = []
while len(forw_cols)<30:
    with Pool(20) as p:
        res = list(tqdm_notebook(p.imap(forw, [i for i in var_lst if i not in forw_cols]), total=len(var_lst)-len(forw_cols), leave=False))
    res = pd.DataFrame({i[0]:i[1] for i in res}).T
    delta = res - current_ginis
    if delta['Test CV'].max()<0:
        break
    best_var = delta['Test CV'].idxmax()
    forw_cols = forw_cols + [best_var]
    current_ginis = res.loc[best_var]
    forwards_log.append(current_ginis)
    clear_output()
    print(pd.DataFrame(forwards_log))

clear_output()
forwards_log = pd.DataFrame(forwards_log)
forwards_log['Uplift Train CV'] = forwards_log['Train CV']-forwards_log['Train CV'].shift(1).fillna(0)
forwards_log['Uplift Test CV'] = forwards_log['Test CV']-forwards_log['Test CV'].shift(1).fillna(0)
print(forwards_log)

              Train CV   Test CV  Uplift Train CV  Uplift Test CV
odor          0.970307  0.843798         0.970307        0.843798
gill-size     0.979971  0.958396         0.009664        0.114598
gill-spacing  0.984611  0.964531         0.004640        0.006135
stalk-shape   0.993921  0.982878         0.009310        0.018347
habitat       0.995208  0.984953         0.001287        0.002075


Select features which have Uplift Test CV > 0.001

In [18]:
ids_vars = forwards_log[forwards_log['Uplift Test CV']>0.001].index.values.tolist()
vars_gini.loc[ids_vars,:]

,gini_train,gini_test
0,,
odor,0.967487,0.969555
gill-size,0.490563,0.535203
gill-spacing,0.256251,0.257858
stalk-shape,0.102960,0.093950
habitat,0.444780,0.476353


In [19]:
mushroom_data_features = mushroom_data[ids_vars + ["classes"]]
mushroom_data = mushroom_data_features.loc[mushroom_data_features.index.repeat(4)].reset_index(drop=True)

mushroom_data["a"] = np.random.choice([0, 1], mushroom_data.shape[0])
mushroom_data["probs"] = 1
mushroom_data["y"] = 0

eat_edible = (1-mushroom_data["classes"]) * mushroom_data["a"] * 1
eat_poisonous = mushroom_data["classes"] * mushroom_data["a"] * np.random.choice([1, -1], mushroom_data.shape[0])
mushroom_data["y"] = eat_edible + eat_poisonous
new_names = ['X_' + str(i+1) for i in range(len(ids_vars))]    
mushroom_data = mushroom_data.rename(columns=dict(zip(ids_vars, new_names)))

mushroom_data_final = mushroom_data[new_names + ['a', 'y', 'probs']]

In [20]:
mushroom_data_final.head()

,X_1,X_2,X_3,X_4,X_5,a,y,probs
0,6,1,0,0,5,0,0,1
1,6,1,0,0,5,1,1,1
2,6,1,0,0,5,1,-1,1
3,6,1,0,0,5,1,-1,1
4,0,0,0,0,1,1,1,1


In [21]:
with open('mushroom_data_final.pickle', 'wb') as handle:
    pickle.dump(mushroom_data_final, handle, protocol=pickle.HIGHEST_PROTOCOL)